<h1>The Generalized Mean Function<h1>

# I. Summary

Both the **generalized_mean** and **generalized_mean_alt** functions are robust to small numbers close to the limits of precision, so we should use the simpler **generalized_mean** function.

# II. Function Descriptions

The cell below holds the **generalized_mean** and **generalized_mean_alt** functions, which depend on the **coupled_logarithm** and **coupled_exponential** functions. They take in 1-D numpy array of non-negative numbers (*values*), a risk-bias parameter (*r*), which must be numeric and is the power of the generalized mean, and a 1-D numpy array of weights (creatively called *weights*) for calculating weighted generalized means. If *weights* equals False, equal weighting of each value is used. The **generalized_mean_alt** function has an additional parameter called *norm_factor* that is discussed more below.

Both functions first calculate the log-generalized mean using the coupled logarithm, and then exponentiates it to calculate the generalized mean.

The log-generalized mean is calculated as: 

### $\text{ln}_{r}\text{(generalized mean)} = \frac{1}{\Sigma^n_{i=1}w_i}\Sigma^{n}_{i=1}w_i\text{ln}_{r}(x_{i})$

The generalized mean is calculated as:

### $\text{generalized mean} = \text{exp}_{r} \text{(ln}_{r} \text{(generalized mean))}$

For the **generalized_mean_alt** function, $\text{ln}_{r}(x_{i})$ is calculated as:

### $\text{log}_{r}(x_{i}) = q \frac{1}{q} \text{log}_{r}(x_{i}) = q \frac{1}{q} \frac{1}{r}(x_{i}^{r}-1) = q \frac{1}{qr}(x_{i}^{r}-1) = q \frac{1}{qr}(x_{i}^{\frac{qr}{q}}-1) = q \text{log}_{qr}(x_{i}^{\frac{1}{q}})$

where $q$ is the normalization factor (*norm_factor*).

The tests for the **generalized_mean** and **generalized_mean_alt** functions are below the cell defining the functions.

In [184]:
import numpy as np
import pandas as pd
import math
from typing import Any, List  # for NDArray types

def generalized_mean(values: np.ndarray, r: float = 1.0, weights: np.ndarray = False) -> float:
    """
    This function calculates the generalized mean of a 1-D array of non- 
    negative real numbers using the coupled logarithm and exponential functions.
    
    Parameters
    ----------
    values : np.ndarray
        DESCRIPTION : A 1-D numpy array (row vector) of non-negative numbers
         for which we are calculating the generalized mean.
    r : float, optional
        DESCRIPTION : The risk bias and the power of the generalized mean. 
        The default is 1.0 (Arithmetric Mean).
    weights : np.ndarray, optional
        DESCRIPTION : A 1-D numpy array of the weights for each value. 
        The default is False, which triggers a conditional to use equal weights.

    Returns gen_mean
    -------
    float
        DESCRIPTION : The coupled generalized mean.
    """
    
    # If weights equals False, equally weight all observations.
    if weights == False:
        # Create an array of all ones equal in length to values.
        weights = np.ones(len(values))
    
    # Calculate the log of the generalized mean by taking the dot product of the
    # weights vector and the vector of the coupled logarithm of the values and
    # divide the result by the sum of the the weights.
    log_gen_mean = np.dot(weights, coupled_logarithm(values, kappa=r, dim=0)) / np.sum(weights)
        
    # Calculate the generalized mean by exponentiating the log-generalized mean.
    gen_mean = coupled_exponential(log_gen_mean, kappa=r, dim=0)
    
    # Return the generalized mean.
    return gen_mean

def generalized_mean_alt(values: np.ndarray, r: float = 1.0, weights: np.ndarray = False, norm_factor: int = 1) -> float:
    """
    This function calculates the generalized mean of a 1-D array of non- 
    negative real numbers using the coupled logarithm and exponential functions.
    
    Parameters
    ----------
    values : np.ndarray
        DESCRIPTION : A 1-D numpy array (row vector) of non-negative numbers
         for which we are calculating the generalized mean.
    r : float, optional
        DESCRIPTION : The risk bias and the power of the generalized mean. 
        The default is 1.0 (Arithmetric Mean).
    weights : np.ndarray, optional
        DESCRIPTION : A 1-D numpy array of the weights for each value. 
        The default is False, which triggers a conditional to use equal weights.
    norm_factor : int
        DESCRIPTION : An integer whose inverse the values array is raised to,
        -r is multiplied by for the coupled_logarithm function call, and then the
        result is multiplied by to calculate the coupled log of the values.
        log_r(values) = norm_factor * log_(norm_factor*r)(values^(1/norm_factor))
        The default is 1.
    
    Returns gen_mean
    -------
    float
        DESCRIPTION : The coupled generalized mean.
    """
    
    # If weights equals False, equally weight all observations.
    if weights == False:
        # Create an array of all ones equal in length to values.
        weights = np.ones(len(values))
    
    # Calculate the log of the generalized mean by taking the dot product of the
    # weights vector and the vector of the coupled logarithm of the values and
    # divide the result by the sum of the the weights.
    coupled_log_values = norm_factor * coupled_logarithm(values**(1/norm_factor), kappa=r*norm_factor, dim=0)
    log_gen_mean = np.dot(weights, coupled_log_values) / np.sum(weights)
        
    # Calculate the generalized mean by exponentiating the log-generalized mean.
    gen_mean = coupled_exponential(log_gen_mean, kappa=r, dim=0)
    
    # Return the generalized mean.
    return gen_mean

def coupled_logarithm(value: [float, Any], kappa: float = 0.0, dim: int = 1) -> [float, Any]:
    """
    Generalization of the logarithm function, which defines smooth
    transition to power functions.
    Inputs
    ----------
    x : Input variable in which the coupled logarithm is applied to.
    kappa : Coupling parameter which modifies the coupled logarithm function.
    dim : The dimension of x, or rank if x is a tensor. Not needed?
    """
    if isinstance(value, float):    
        assert value >= 0, "x must be greater or equal to 0."  # Greater than 0?????
    else:
        assert isinstance(value, np.ndarray), "x must be a np.ndarray type if a sequence, or a float if a scalar."
        # assert np.all(x), "all values in x must be "

    if kappa == 0:
        coupled_log_value = np.log(value)  # divide by 0 if x == 0
    else:
        coupled_log_value = (1 / kappa) * (value**(kappa / (1 + dim*kappa)) - 1)
    return coupled_log_value


def coupled_exponential(value: float, kappa: float = 0.0, dim: int = 1) -> float:
    """
    Short description
    ----------
    x : Input variable in which the coupled exponential is applied to.
    kappa : Coupling parameter which modifies the coupled exponential function.
    dim : The dimension of x, or rank if x is a tensor.
    """
    assert dim >= 0, "dim must be greater than or equal 0."
        # may also want to test that dim is an integer

        # removed the requirement on kappa; although not common kappa can be less than -1/dim
    if kappa == 0:
        coupled_exp_value = math.exp(value)
    else:
        if kappa > 0:
        	coupled_exp_value = (1 + kappa*value)**(1/(kappa / (1 + dim*kappa))) # removed negative sign and added reciprocal
        # now given that kappa < 0
        elif (1 + kappa*value) >= 0:
       		coupled_exp_value = (1 + kappa*value)**(1/(kappa / (1 + dim*kappa))) # removed negative sign and added reciprocal
        elif (kappa / (1 + dim*kappa)) > 0: # removed negative sign
       		coupled_exp_value = 0
        else:
       		coupled_exp_value = float('inf')
        # else:
        # 	print("Error: kappa = 1/d is not greater than -1.")
    return coupled_exp_value

# III. Quick Tests

To test the **generalized_mean** function, I tested the harmonic ($p = -1$), $-\frac{2}{3}$ ($p = -\frac{2}{3}$), geometric ($p = 0$), and arithmetic ($p = 1$) means. For the harmonic, geometric, and arithmetic means, I compared the outputs of the **generalized_mean** and **generalized_mean_alt** functions to built-in functions in numpy and scipy.

Each of the four generalized means were tested on the random vectors $\vec{X} = [X_1, X_2, ..., X_n]$, where $X_i \overset{\text{iid}}{\sim} \text{N }(100, 25)$ and $\vec{Y} = [Y_1, Y_2, ..., Y_n]$, where $Y_i \overset{\text{iid}}{\sim} \text{Beta }(0.0001, 0.1)$, for $i \in \{1, 2, ..., n\}$ and $n=1,000,000$. A random seed is used, so the functions are tested on realizations of the random vectors. They are now referred to as $\vec{x}$ and $\vec{y}$. For $\vec{y}$, I had to filter out numbers too small for double precision floats and were set to $0$. Of the $1,000,000$ original values, $71,602$ values passed this filter.

$\vec{x}$ is meant to compare the functions on numbers that are not very small or large. $\vec{y}$ is meant to compare the functions on very small numbers that are not too small for the 64 bit float data type. Smaller numbers might require more memory to store.

On both $\vec{x}$ and $\vec{y}$, the **generalized_mean** and **generalized_mean_alt** functions give answers very close to the built-in functions for the harmonic, geometric, and arithmetic means.

In [185]:
# Import the function to calculate harmonic means from a scipy module.
from scipy.stats.mstats import hmean
# Import the function to calculate geometric means from a scipy module.
from scipy.stats.mstats import gmean

# Set the size of each numpy array of pseudo-random numbers.
n = 1000000

# Set a random seed for reproducibility.
np.random.seed(0)

# Generate n pseudo-random numbers from a N(100, 25) distribution.
x = np.random.normal(loc=100, scale=5, size=n)
# Generate n pseudo-random numbers from a Beta(0.01, 1) distribution.
y = np.random.beta(0.0001, 0.1, size=n)
# Filter out the y-values so small they equal 0.
y = y[y != 0]


print(f'{y.shape[0]:,} values of {n:,} y values are large enough to be represented by a double precision float and remain in the array.')
# Display 10 values of y.
pd.DataFrame(y, columns=['y']).head(10)

71,602 values of 1,000,000 y values are large enough to be represented by a double precision float and remain in the array.


,y
0,9.120758e-90
1,5.759064e-109
2,1.236640e-133
3,3.793776e-282
4,8.535806e-237
5,1.971696e-28
6,1.357865e-28
7,2.685447e-253
8,2.651561e-123
9,2.462258e-254


## a) Harmonic Mean ($p = -1$)

### i) $\vec{x}$

In [186]:
a = generalized_mean(x, -1, weights=False)
b = generalized_mean_alt(x, -1, weights=False, norm_factor=100)
c = hmean(x)

if np.isclose(a, b) & np.isclose(b, c):
    d = 'Yes'
else:
    d = 'No'
    
print(f'The output from the generalized_mean function is {a}.', 
      f'\nThe output from the generalized_mean_alt function is {b}.',
      f'\nThe output from the scipy/numpy function is {c}.',
      f'\nAre these reasonably close? {d}.')

The output from the generalized_mean function is 99.75633669505517. 
The output from the generalized_mean_alt function is 99.75633669505517. 
The output from the scipy/numpy function is 99.7563366950605. 
Are these reasonably close? Yes.


### ii) $\vec{y}$

In [187]:
a = generalized_mean(y, -1, weights=False)
b = generalized_mean_alt(y, -1, weights=False, norm_factor=100)
c = hmean(y)

if np.isclose(a, b) & np.isclose(b, c):
    d = 'Yes'
else:
    d = 'No'
    
print(f'The output from the generalized_mean function is {a}.', 
      f'\nThe output from the generalized_mean_alt function is {b}.',
      f'\nThe output from the scipy/numpy function is {c}.',
      f'\nAre these reasonably close? {d}.')

The output from the generalized_mean function is 0.0. 
The output from the generalized_mean_alt function is 0.0. 
The output from the scipy/numpy function is 0.0. 
Are these reasonably close? Yes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: RuntimeWarning: overflow encountered in reciprocal
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: RuntimeWarning: overflow encountered in power
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:404: RuntimeWarning: overflow encountered in true_divide
  return size / np.sum(1.0 / a, axis=axis, dtype=dtype)
/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


## b) $-\frac{2}{3}$ Mean ($p = -\frac{2}{3}$)

There is not $-\frac{2}{3}$ mean function in numpy or scipy to compare to, so I just compared the two generalized mean functions.

### i) $\vec{x}$

In [188]:
a = generalized_mean(x, -2/3, weights=False)
b = generalized_mean_alt(x, -2/3, weights=False, norm_factor=100)

if np.isclose(a, b):
    c = 'Yes'
else:
    c = 'No'
    
print(f'The output from the generalized_mean function is {a}.', 
      f'\nThe output from the generalized_mean_alt function is {b}.',
      f'\nAre these reasonably close? {c}.')

The output from the generalized_mean function is 99.79834197665035. 
The output from the generalized_mean_alt function is 99.7983419766507. 
Are these reasonably close? Yes.


### ii) $\vec{y}$

In [189]:
a = generalized_mean(y, -2/3, weights=False)
b = generalized_mean_alt(y, -2/3, weights=False, norm_factor=100)

if np.isclose(a, b):
    c = 'Yes'
else:
    c = 'No'
    
print(f'The output from the generalized_mean function is {a}.', 
      f'\nThe output from the generalized_mean_alt function is {b}.',
      f'\nAre these reasonably close? {c}.')

The output from the generalized_mean function is 5.4621e-319. 
The output from the generalized_mean_alt function is 5.4621e-319. 
Are these reasonably close? Yes.


## c) Geometric Mean ($p = 0$)

### i) $\vec{x}$

In [190]:
a = generalized_mean(x, 0, weights=False)
b = generalized_mean_alt(x, 0, weights=False, norm_factor=100)
c = gmean(x)

if np.isclose(a, b) & np.isclose(b, c):
    d = 'Yes'
else:
    d = 'No'
    
print(f'The output from the generalized_mean function is {a}.', 
      f'\nThe output from the generalized_mean_alt function is {b}.',
      f'\nThe output from the scipy/numpy function is {c}.',
      f'\nAre these reasonably close? {d}.')

The output from the generalized_mean function is 99.88219035772059. 
The output from the generalized_mean_alt function is 99.88219035772059. 
The output from the scipy/numpy function is 99.88219035772121. 
Are these reasonably close? Yes.


### ii) $\vec{y}$

In [191]:
a = generalized_mean(y, 0, weights=False)
b = generalized_mean_alt(y, 0, weights=False, norm_factor=100)
c = gmean(y)

if np.isclose(a, b) & np.isclose(b, c):
    d = 'Yes'
else:
    d = 'No'
    
print(f'The output from the generalized_mean function is {a}.', 
      f'\nThe output from the generalized_mean_alt function is {b}.',
      f'\nThe output from the scipy/numpy function is {c}.',
      f'\nAre these reasonably close? {d}.')

The output from the generalized_mean function is 3.76877219250309e-156. 
The output from the generalized_mean_alt function is 3.76877219250309e-156. 
The output from the scipy/numpy function is 3.7687721925033044e-156. 
Are these reasonably close? Yes.


## d) Arithmetic Mean ($p = 1$)

### i) $\vec{x}$

In [192]:
a = generalized_mean(x, 1, weights=False)
b = generalized_mean_alt(x, 1, weights=False, norm_factor=100)
c = np.mean(x)

if np.isclose(a, b) & np.isclose(b, c):
    d = 'Yes'
else:
    d = 'No'
    
print(f'The output from the generalized_mean function is {a}.', 
      f'\nThe output from the generalized_mean_alt function is {b}.',
      f'\nThe output from the scipy/numpy function is {c}.',
      f'\nAre these reasonably close? {d}.')

The output from the generalized_mean function is 100.00756073257767. 
The output from the generalized_mean_alt function is 100.00756073257769. 
The output from the scipy/numpy function is 100.00756073257772. 
Are these reasonably close? Yes.


### ii) $\vec{y}$

In [193]:
a = generalized_mean(y, 1, weights=False)
b = generalized_mean_alt(y, 1, weights=False, norm_factor=100)
c = np.mean(y)

if np.isclose(a, b) & np.isclose(b, c):
    d = 'Yes'
else:
    d = 'No'
    
print(f'The output from the generalized_mean function is {a}.', 
      f'\nThe output from the generalized_mean_alt function is {b}.',
      f'\nThe output from the scipy/numpy function is {c}.',
      f'\nAre these reasonably close? {d}.')

The output from the generalized_mean function is 0.013931843947880829. 
The output from the generalized_mean_alt function is 0.013931843947880829. 
The output from the scipy/numpy function is 0.01393184394788079. 
Are these reasonably close? Yes.


## e) Final Tests

I created a numpy array of $4.95 * 10^{-323}$ repeated 1,000,000 times. $4.95 * 10^{-323}$ is 1 order of magnitude from being as small as a number that can be represented with a double precision float. I then multiplied each element in that vector of repeats by $z_i$, where $Z \sim N(10, 1)$ to generate random numbers close to the maximum precision. Neither function threw an error trying to calculate the $\frac{-2}{3}$ mean, so it seems like the **generalized_mean_alt** function is not needed and the simpler **generalized_mean** function is what we should use.

### i) On 1,000,000 Small Random Numbers Close to the 64-bit Precision Limit

In [194]:
n = 1000000

z = np.repeat(4.95*10**(-323), n) * np.random.normal(loc=10, scale=1, size=n)**2

generalized_mean(z, r=-2/3, weights=False), generalized_mean_alt(z, r=-2/3, weights=False, norm_factor=100)

(4.82e-321, 4.82e-321)

### ii) On 100,000 Small and Large Numbers at the limit of 64-bit Precision

In [195]:
float_info = np.finfo(np.float64)
print(f'Largest Number: {float_info.max}')
smallest_num = float_info.tiny
print(f'Smallest Number: {smallest_num}')

Largest Number: 1.7976931348623157e+308
Smallest Number: 2.2250738585072014e-308


In [196]:
n = 100000

In [197]:
smallest = np.repeat(smallest_num, n)

print('generalized_mean')
print(generalized_mean(smallest, -1), 
      generalized_mean(smallest, -2/3), 
      generalized_mean(smallest, 0), 
      generalized_mean(smallest, 1))
print('\ngeneralized_mean_alt')
print(generalized_mean_alt(smallest, -1, norm_factor=100), 
      generalized_mean_alt(smallest, -2/3, norm_factor=100),
      generalized_mean_alt(smallest, 0, norm_factor=100),
      generalized_mean_alt(smallest, 1, norm_factor=100))

generalized_mean
0.0 2.2250738585072256e-308 2.2250738584167024e-308 0.0

generalized_mean_alt
0.0 2.2250738585073037e-308 2.2250738584167024e-308 0.0


In [198]:
smallest = np.repeat(smallest_num*(10**5), n)

print('generalized_mean')
print(generalized_mean(smallest, -1), 
      generalized_mean(smallest, -2/3), 
      generalized_mean(smallest, 0), 
      generalized_mean(smallest, 1))
print('\ngeneralized_mean_alt')
print(generalized_mean_alt(smallest, -1, norm_factor=100), 
      generalized_mean_alt(smallest, -2/3, norm_factor=100),
      generalized_mean_alt(smallest, 0, norm_factor=100),
      generalized_mean_alt(smallest, 1, norm_factor=100))

generalized_mean
2.22507385850728e-303 2.2250738585072512e-303 2.2250738585529423e-303 0.0

generalized_mean_alt
2.225073858507255e-303 2.2250738585072797e-303 2.2250738585529423e-303 0.0


In [199]:
smallest = np.repeat(smallest_num*(10**292), n)

print('generalized_mean')
print(generalized_mean(smallest, -1), 
      generalized_mean(smallest, -2/3), 
      generalized_mean(smallest, 0), 
      generalized_mean(smallest, 1))
print('\ngeneralized_mean_alt')
print(generalized_mean_alt(smallest, -1, norm_factor=100), 
      generalized_mean_alt(smallest, -2/3, norm_factor=100),
      generalized_mean_alt(smallest, 0, norm_factor=100),
      generalized_mean_alt(smallest, 1, norm_factor=100))

generalized_mean
2.2250738585073513e-16 2.225073858507366e-16 2.225073858501767e-16 1.1102230246251565e-16

generalized_mean_alt
2.22507385850735e-16 2.225073858507355e-16 2.225073858501767e-16 1.1102230246251565e-16


In [200]:
largest = np.repeat(1.7976931348623157e+308, n)

print('generalized_mean')
print(generalized_mean(largest, -1), 
      generalized_mean(largest, -2/3), 
      generalized_mean(largest, 0), 
      generalized_mean(largest, 1))
print('\ngeneralized_mean_alt')
print(generalized_mean_alt(largest, -1, norm_factor=100), 
      generalized_mean_alt(largest, -2/3, norm_factor=100),
      generalized_mean_alt(largest, 0, norm_factor=100),
      generalized_mean_alt(largest, 1, norm_factor=100))

generalized_mean
inf inf 1.7976931348087272e+308 inf

generalized_mean_alt
inf inf 1.7976931348087272e+308 inf


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:133: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:110: RuntimeWarning: overflow encountered in power


In [201]:
largest = np.repeat(1.7976931348623157e+303, n)

print('generalized_mean')
print(generalized_mean(largest, -1), 
      generalized_mean(largest, -2/3), 
      generalized_mean(largest, 0), 
      generalized_mean(largest, 1))
print('\ngeneralized_mean_alt')
print(generalized_mean_alt(largest, -1, norm_factor=100), 
      generalized_mean_alt(largest, -2/3, norm_factor=100),
      generalized_mean_alt(largest, 0, norm_factor=100),
      generalized_mean_alt(largest, 1, norm_factor=100))

generalized_mean
inf inf 1.7976931348952634e+303 1.7976931348622524e+303

generalized_mean_alt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:133: RuntimeWarning: divide by zero encountered in double_scalars


inf inf 1.7976931348952634e+303 1.7976931348622725e+303


In [202]:
largest = np.repeat(1.7976931348623157e+24, n)

print('generalized_mean')
print(generalized_mean(largest, -1), 
      generalized_mean(largest, -2/3), 
      generalized_mean(largest, 0), 
      generalized_mean(largest, 1))
print('\ngeneralized_mean_alt')
print(generalized_mean_alt(largest, -1, norm_factor=100), 
      generalized_mean_alt(largest, -2/3, norm_factor=100),
      generalized_mean_alt(largest, 0, norm_factor=100),
      generalized_mean_alt(largest, 1, norm_factor=100))

generalized_mean
inf 3.022314549036573e+23 1.7976931348667672e+24 1.7976931348623925e+24

generalized_mean_alt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:133: RuntimeWarning: divide by zero encountered in double_scalars


inf inf 1.7976931348667672e+24 1.7976931348623962e+24


In [203]:
largest = np.repeat(1.7976931348623157e+23, n)

print('generalized_mean')
print(generalized_mean(largest, -1), 
      generalized_mean(largest, -2/3), 
      generalized_mean(largest, 0), 
      generalized_mean(largest, 1))
print('\ngeneralized_mean_alt')
print(generalized_mean_alt(largest, -1, norm_factor=100), 
      generalized_mean_alt(largest, -2/3, norm_factor=100),
      generalized_mean_alt(largest, 0, norm_factor=100),
      generalized_mean_alt(largest, 1, norm_factor=100))

generalized_mean
inf 3.022314549036573e+23 1.797693134856093e+23 1.7976931348622298e+23

generalized_mean_alt


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:133: RuntimeWarning: divide by zero encountered in double_scalars


inf 3.022314549036573e+23 1.797693134856093e+23 1.79769313486223e+23


In [204]:
largest = np.repeat(1.7976931348623157e+16, n)

print('generalized_mean')
print(generalized_mean(largest, -1), 
      generalized_mean(largest, -2/3), 
      generalized_mean(largest, 0), 
      generalized_mean(largest, 1))
print('\ngeneralized_mean_alt')
print(generalized_mean_alt(largest, -1, norm_factor=100), 
      generalized_mean_alt(largest, -2/3, norm_factor=100),
      generalized_mean_alt(largest, 0, norm_factor=100),
      generalized_mean_alt(largest, 1, norm_factor=100))

generalized_mean
9007199254740992.0 1.8008627614458812e+16 1.7976931348664958e+16 1.7976931348621944e+16

generalized_mean_alt
9007199254740992.0 1.8008627614458812e+16 1.7976931348664958e+16 1.7976931348621924e+16
